<a href="https://colab.research.google.com/github/Arindam75/LargeLanguageModels/blob/main/LLM_FineTuning_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Packages

In [4]:
!pip install transformers tokenizers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [18]:
from google.colab import drive
drive.mount('/content/drive')
import os,sys
import json

root_path = "/content/drive/MyDrive/LLM"
f = open(os.path.join(root_path,"keyfile.json"))
key_data = json.load(f)
hf_key = key_data['my_keys'][1]['hf']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from datasets import load_dataset
from transformers import AutoTokenizer

## Load Dataset

We load the squad dataset, which is of type __datasets.dataset_dict.DatasetDict__. Once downloaded , we can see that it has 4 key value pairs.

- id
- context
- question
- answers

We can perform a train test split and check the first training instance.

In [20]:
squad = load_dataset("squad", split="train[:5000]")

In [21]:
squad = squad.train_test_split(test_size=0.2)
squad["train"][0]

{'id': '56cd7ef762d2951400fa6656',
 'title': 'IPod',
 'context': 'iPods have been criticized for alleged short life-span and fragile hard drives. A 2005 survey conducted on the MacInTouch website found that the iPod line had an average failure rate of 13.7% (although they note that comments from respondents indicate that "the true iPod failure rate may be lower than it appears"). It concluded that some models were more durable than others. In particular, failure rates for iPods employing hard drives was usually above 20% while those with flash memory had a failure rate below 10%. In late 2005, many users complained that the surface of the first generation iPod Nano can become scratched easily, rendering the screen unusable. A class action lawsuit was also filed. Apple initially considered the issue a minor defect, but later began shipping these iPods with protective sleeves.[citation needed]',
 'question': 'What did Apple include with the iPod to resolve the screen problem?',
 'answers

There are several important fields here:

answers: the starting location of the answer token and the answer text.<br>
context: background information from which the model needs to extract the answer.<br>
question: the question a model should answer.

## Tokenization

In [23]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

1. Some examples in a dataset may have a very long context that exceeds the maximum input length of the model. To deal with longer sequences, truncate only the context by setting truncation="only_second".
2. A Note from [huggingface](https://huggingface.co/docs/transformers/en/pad_truncation). Truncate to a maximum length specified by the max_length argument or the maximum length accepted by the model if no max_length is provided (max_length=None). This will only truncate the second sentence of a pair if a pair of sequences (or a batch of pairs of sequences) is provided.
3. Next, map the start and end positions of the answer to the original context by setting return_offset_mapping=True.
4. With the mapping in hand, now you can find the start and end tokens of the answer. Use the sequence_ids method to find which part of the offset corresponds to the question and which corresponds to the context.

In [16]:
type(squad)

datasets.dataset_dict.DatasetDict